
# QLD Sensitive List

## List Identifiers
* Conservation list: __[dr652](https://lists.ala.org.au/speciesListItem/list/dr652)__
* Sensitive list: __[dr493](https://lists.ala.org.au/speciesListItem/list/dr493)__

### Sources
Queensland Nature Conservation Act 1992

## Conservation
* __[Metadata - Qld Species (Open Data Portal)](https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife)__
* __[Data](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species.csv)__

## Sensitive
* __[Metadata - Queensland Confidential Species (Open Data Portal)](https://www.data.qld.gov.au/dataset/queensland-confidential-species)__
* __[Data](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/qld-confidential-species.csv)__

## Codes
* __[Metadata - Qld Species codes](https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife/resource/6344ea93-cadf-4e0c-9ff4-12dfb18d5f14)__
* __[Data](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv)__


Import libraries
Set Project directory

In [ ]:
import pandas as pd
import requests
import io
from ftfy import fix_encoding

projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/"

## Download Qld Species Status Codes

In [ ]:
codesurl =  "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv"
print("Downloading Qld Species Status Codes")
response = requests.get(codesurl)
rtext = fix_encoding(response.text)
speciescodes = pd.read_csv(io.StringIO(rtext))

Firstly, some minimal changes to some Qld Nature Conservation Act codes so that they are consistent with other states

In [ ]:
ncastatuscodes = speciescodes[speciescodes['Field'] == "NCA_status"][['Code', 'Code_description']]
ncastatuscodes['Code_description'] = ncastatuscodes['Code_description'].str.replace(" wildlife", "")
ncastatuscodes.loc[
ncastatuscodes['Code_description'] == "Critically endangered", 'Code_description'] = "Critically Endangered"
ncastatuscodes.loc[ncastatuscodes['Code_description'] == "Near threatened", 'Code_description'] = "Near Threatened"
endemicitycodes = speciescodes[speciescodes['Field'] == "Endemicity"][['Code', 'Code_description']]
# ebpc codes
epbccodes = speciescodes[speciescodes['Field'] == "EPBC_status"][['Code', 'Code_description']]

## Process Conservation List

Set species list CSV file URLs

In [ ]:
listurl = "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species.csv"

Files had encoding errors that require update
Read in the Sensitive list and join to get the code descriptions.
Change the field names to sourceStatus and status as required by the ALA’s list processing.

In [ ]:
print("Downloading Qld Conservation List")
response = requests.get(listurl)
# Remove bad encoding
rtext = fix_encoding(response.text)

Read in the Conservation list and join to get the code descriptions.
Change the field names to `sourceStatus` and `status` as required by the ALA's conservation list processing.

In [ ]:
conservationlist = pd.read_csv(io.StringIO(rtext))
conservationlist = pd.merge(conservationlist,ncastatuscodes,left_on=['NCA_status'],right_on=['Code'],how="left")
conservationlist.drop(['Code'],axis=1,inplace=True)
conservationlist = conservationlist.rename(columns={'NCA_status':'sourceStatus','Code_description':'status'})
conservationlist

In [ ]:
endemicitycodes = speciescodes[speciescodes['Field'] == "Endemicity"][['Code','Code_description']]
endemicitycodes

In [ ]:
endemicitycodes = speciescodes[speciescodes['Field'] == "Endemicity"][['Code','Code_description']]
conservationlist = pd.merge(conservationlist,endemicitycodes,left_on=['Endemicity'],right_on=['Code'],how="left")
conservationlist.drop(['Code'],axis=1,inplace=True)
conservationlist = conservationlist.rename(columns={'Code_description':'Endemicity_description'})
conservationlist

Expand EPBC codes and merge data

In [ ]:
epbccodes = speciescodes[speciescodes['Field'] == "EPBC_status"][['Code','Code_description']]
conservationlist = pd.merge(conservationlist,epbccodes,left_on=['EPBC_status'],right_on=['Code'],how="left")
conservationlist.drop(['Code'],axis=1,inplace=True)
conservationlist = conservationlist.rename(columns={'Code_description':'EPBC_status_description'})
conservationlist.drop(['EPBC_status'],axis=1,inplace=True)

**Species without a status?**

In [ ]:
conservationlist[conservationlist['status'].isna()].groupby(["Class","Endemicity_description"]).size()

**Tidy up**
* rename fields to Darwin Core
* remove records with no status or Least Concern
* replace kingdom/class values with scientific names

In [ ]:
conservationlist = conservationlist.rename(columns=
{
    'Taxon_Id':'taxonID',
    'Kingdom':'kingdom',
    'Class':'class',
    'Family':'family',
    'Scientific_name':'scientificName',
    'Common_name': 'vernacularName',
    'Taxon_author':'scientificNameAuthorship',
    'NCA_status':'sourceStatus',
    'EPBC_status_description':'epbcStatus',
    'Endemicity_description':'endemicity',
    'Significant':'significant',
    'Confidential':'confidential'
})
# remove empty or Least Concern status records
conservationlist = conservationlist[((conservationlist['status'] != "Least concern") & (conservationlist['status'].notna()))]

# Replace kingdom and class values with scientific terms
conservationlist.loc[conservationlist["kingdom"] == "animals", "kingdom"] = "Animalia"
conservationlist.loc[conservationlist["kingdom"] == "plants", "kingdom"] = "Plantae"
conservationlist.loc[conservationlist["class"] == "land plants", "class"] = "Equisetopsida"
conservationlist.loc[conservationlist["class"] == "amphibians", "class"] = "Amphibia"
conservationlist.loc[conservationlist["class"] == "birds", "class"] = "Aves"
conservationlist.loc[conservationlist["class"] == "cartilaginous fishes", "class"] = "Chondrichthyes"
conservationlist.loc[conservationlist["class"] == "insects", "class"] = "Insecta"
conservationlist.loc[conservationlist["class"] == "malacostracans", "class"] = "Malacostraca"
conservationlist.loc[conservationlist["class"] == "mammals", "class"] = "Mammalia"
conservationlist.loc[conservationlist["class"] == "ray-finned fishes", "class"] = "Actinopterygii"
conservationlist.loc[conservationlist["class"] == "reptiles", "class"] = "Reptilia"
conservationlist.loc[conservationlist["class"] == "snails", "class"] = "Gastropoda"
conservationlist.loc[conservationlist["class"] == "arachnids", "class"] = "Arachnida"
conservationlist.groupby(["kingdom","class"]).size()

Write dataframe to CSV - UTF-8 encoding

In [ ]:
print("Writing to CSV")
conservationlist.to_csv(projectdir + "current-lists/conservation-lists/QLD-conservation-2022-10.csv",encoding="UTF-8",index=False)
print("Processing finished")

## Process Sensitive List

Set species list CSV file URLs

In [ ]:
#Species List and Species codes URLS
listurl = "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/qld-confidential-species.csv"


Files had encoding errors that require update
Read in the Sensitive list and join to get the code descriptions.
Change the field names to sourceStatus and status as required by the ALA’s list processing.

In [ ]:
print("Downloading Qld Sensitive List")
response = requests.get(listurl)
rtext = fix_encoding(response.text)
confidentiallist = pd.read_csv(io.StringIO(rtext))
confidentiallist

In [ ]:
# nca status
confidentiallist = pd.merge(confidentiallist,ncastatuscodes,left_on=['NCA status'],right_on=['Code'],how="left")
confidentiallist.drop(['Code'],axis=1,inplace=True)
confidentiallist = confidentiallist.rename(columns={'NCA status':'sourceStatus','Code_description':'status'})
# endemicity
confidentiallist = pd.merge(confidentiallist,endemicitycodes,left_on=['Endemicity'],right_on=['Code'],how="left")
confidentiallist.drop(['Code','Endemicity'],axis=1,inplace=True)
confidentiallist = confidentiallist.rename(columns={'Code_description':'endemicity'})
# epbc
confidentiallist = pd.merge(confidentiallist,epbccodes,left_on=['EPBC status'],right_on=['Code'],how="left")
confidentiallist.drop(['Code','EPBC status'],axis=1,inplace=True)
confidentiallist = confidentiallist.rename(columns={'Code_description':'epbcStatus'})
# rename fields
confidentiallist = confidentiallist.rename(columns=
{
    'Taxon Id':'taxonID',
    'Kingdom':'kingdom',
    'Class':'class',
    'Family':'family',
    'Scientific name':'scientificName',
    'Common name': 'vernacularName',
    'Taxon author':'scientificNameAuthorship',
    'Confidential':'confidential',
    'Significant':'significant'
})
confidentiallist

Replace kingdom and class values with scientific terms

In [ ]:
confidentiallist.groupby(["kingdom","class"]).size()

confidentiallist.loc[confidentiallist["kingdom"] == "animals", "kingdom"] = "Animalia"
confidentiallist.loc[confidentiallist["kingdom"] == "plants", "kingdom"] = "Plantae"
confidentiallist.loc[confidentiallist["class"] == "land plants", "class"] = "Equisetopsida"
confidentiallist.loc[confidentiallist["class"] == "amphibians", "class"] = "Amphibia"
confidentiallist.loc[confidentiallist["class"] == "birds", "class"] = "Aves"
confidentiallist.loc[confidentiallist["class"] == "cartilaginous fishes", "class"] = "Chondrichthyes"
confidentiallist.loc[confidentiallist["class"] == "insects", "class"] = "Insecta"
confidentiallist.loc[confidentiallist["class"] == "malacostracans", "class"] = "Malacostraca"
confidentiallist.loc[confidentiallist["class"] == "mammals", "class"] = "Mammalia"
confidentiallist.loc[confidentiallist["class"] == "ray-finned fishes", "class"] = "Actinopterygii"
confidentiallist.loc[confidentiallist["class"] == "reptiles", "class"] = "Reptilia"
confidentiallist.loc[confidentiallist["class"] == "snails", "class"] = "Gastropoda"
confidentiallist.loc[confidentiallist["class"] == "arachnids", "class"] = "Arachnida"

confidentiallist.groupby(["kingdom","class"]).size()

Write dataframe to CSV - UTF-8 encoding

In [ ]:
#Write to CSV
print("Writing to CSV")
confidentiallist.to_csv(projectdir + "current-lists/sensitive-lists/QLD-sensitive-2022-10.csv",encoding="UTF-8",index=False)
print("Processing finished")